In [1]:
import pymssql
import numpy as np
import pandas as pd

In [2]:
# query상에서 start_date & end_date 설정
start_date = "'2019-01-01'"
end_date = "'2019-12-31'"

### 추출 시 사용되는 테이블은 크게 총 4개(4개 테이블 외에도 있지만 맵핑이나 기타 다른 목적을 위해 사용됨)
- 핵심 테이블은 V_RAW_D_학습, V_RAW_M

1. MembershiP_membershipInformation: System ID - User ID 맵핑
2. TBL_APP_Sales_Order: 유효 회원 구분
3. V_RAW_D_학습: 학습 타입 및 유저 추출
4. V_RAW_M: 학습 결과 추출

# 0. 데이터 추출

## 1. V_RAW_D_학습: phase 1

### 1.1 connect to DB

In [3]:
# MSSQL connection
conn = pymssql.connect(host=r"",
                       user='',
                       password='',
                       database='')

# Connection으로부터 Cursor 생성
cursor = conn.cursor()

# set query
query = "SELECT B.UserId, A.메뉴명, A.타입, A.[강의 타입], A.L_Type FROM TABLE AS A INNER JOIN DB.dbo.TABLE AS B ON LOWER(A.ID)  = B.UserName AND 날짜 BETWEEN {} AND {}".format(start_date, end_date)

cursor.execute(query)

# to define columns name
col_names_list = []

for i in range(len(cursor.description)):
    col_names_list.append(cursor.description[i][0])
    
# set query
query = "SELECT B.UserId, A.메뉴명, A.타입, A.[강의 타입], A.L_Type FROM DB.dbo.TABLE AS A INNER JOIN DB.dbo.TABLE AS B ON LOWER(A.ID)  = B.UserName AND 날짜 BETWEEN {} AND {}".format(start_date, end_date)

### 1.2 read to data

In [4]:
# read to data
v_raw_d_study = pd.DataFrame(list(cursor.fetchall()), columns=col_names_list)

# change to string type: 'UserId' UUID -> string
v_raw_d_study['UserId'] = v_raw_d_study['UserId'].astype(str).str.upper().values

## 2. Membership_MembershipInformation

### 2.1 connect to DB

In [5]:
# MSSQL connection
conn = pymssql.connect(host=r"",
                       user='',
                       password='',
                       database='')

# Connection으로부터 Cursor 생성
cursor = conn.cursor()

# set query
query = "SELECT UserId, Gender, BirthDay, Address1 FROM DB.dbo.TABLE"

cursor.execute(query)

# to define columns name
col_names_list = []

for i in range(len(cursor.description)):
    col_names_list.append(cursor.description[i][0])
    
# set query
query = "SELECT UserId, Gender, BirthDay, Address1 FROM DB.dbo.TABLE"

In [6]:
# read to data
memb_info = pd.DataFrame(list(cursor.fetchall()), columns=col_names_list)
memb_info = memb_info.sort_values(by='UserId')

# change to string type: 'UserId' UUID -> string
memb_info['UserId'] = memb_info['UserId'].astype(str).str.upper().values

In [7]:
# V_RAW_D_학습 테이블에서 고려한 기간(start_date & end_date)과 학습 이력이 남겨진 학생을 기준으로 memb_info slicing:
memb_info = memb_info[memb_info['UserId'].isin(v_raw_d_study['UserId'].unique())].reset_index(drop=True)

### 2.2 preprocessing

#### 2.2.1 수도권 여부 판별
- 통상적으로 수도권 정의는 다음과 같음: 서울특별시, 인천광역시, 경기도(군사 분계선 이북 지역은 제외)
- 그러나, 여기서는 경기도 부분의 세부 지역까지 고려하지 않음 즉, 경기도 전체를 수도권으로 봄

In [8]:
# 수도권 list
capital_area_list = ['서울특별시', '인천광역시', '경기도']

# 수도권 여부 컬럼 생성
memb_info['capital_area'] = None

idx = 0

for adress in memb_info['Address1']:
    try:
        city = adress.split(' ')[0]
        
        if city in capital_area_list:
            memb_info['capital_area'][idx] = 1 # 수도권 O
        else:
            memb_info['capital_area'][idx] = 0 # 수도권 X
            
    # cuase by split
    except AttributeError:
        print("###### [AttributeError] where the occurred:", idx)
    
    idx += 1

###### [AttributeError] where the occurred: 5288
###### [AttributeError] where the occurred: 8582
###### [AttributeError] where the occurred: 10518
###### [AttributeError] where the occurred: 13531
###### [AttributeError] where the occurred: 15865
###### [AttributeError] where the occurred: 17210
###### [AttributeError] where the occurred: 19146
###### [AttributeError] where the occurred: 28170
###### [AttributeError] where the occurred: 28981
###### [AttributeError] where the occurred: 30456
###### [AttributeError] where the occurred: 30780
###### [AttributeError] where the occurred: 31491
###### [AttributeError] where the occurred: 32386
###### [AttributeError] where the occurred: 36232
###### [AttributeError] where the occurred: 41595
###### [AttributeError] where the occurred: 45529
###### [AttributeError] where the occurred: 48272
###### [AttributeError] where the occurred: 48648
###### [AttributeError] where the occurred: 52691
###### [AttributeError] where the occurred: 57187
##

## 3. TBL_APP_Sales_Order

### 3.1 connect to DB

In [9]:
# MSSQL connection
conn = pymssql.connect(host=r"",
                       user='',
                       password='',
                       database='')

# Connection으로부터 Cursor 생성
cursor = conn.cursor()

# set query
query = "SELECT A.UserID FROM DB.dbo.TABLE AS A LEFT JOIN DB.dbo.TABLE AS B ON A.OrderNo = B.OrderNo AND B.Status = '33' AND B.CancelDate < '2019-12-31 23:59:59.997' WHERE RegDate <= {} and ExpDate >= {} AND FreeYN = 'N' AND B.OrderNo IS NULL".format(start_date, end_date)

cursor.execute(query)

In [10]:
# read to data: 유효회원의 UserID만 추출
charge_user_list = list(cursor.fetchall())

# UUID to string
charge_user_list = pd.Series(charge_user_list).astype(str)

In [11]:
# slicing (UUID('a071604f-9803-4d89-a5c2-b8305f533f2a'),) to a071604f-9803-4d89-a5c2-b8305f533f2a
for i in range(len(charge_user_list)):
    tmp = str(charge_user_list[i])[7:43]
    tmp = tmp.upper()
    
    charge_user_list[i] = tmp

In [12]:
memb_info['charge_memb'] = 0 # non-charged

# charge_user_list를 memb_info에서 userid cheking 후 1로 변환
memb_info.loc[memb_info['UserId'].isin(charge_user_list), 'charge_memb'] = 1 # charged

## 1. V_RAW_D_학습: phase 2

### 1.3 메뉴명별 수 추출

In [13]:
menu_df = pd.DataFrame(v_raw_d_study.groupby(['UserId', '메뉴명'])['UserId'].count()).rename(columns={'UserId':'count'})
menu_df.reset_index(level=['UserId', '메뉴명'], inplace=True)
menu_df

,UserId,메뉴명,count
0,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,매일 보는 동화,59
1,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,매일학교공부,196
2,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,수준별수학,105
3,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,수준별영어,123
4,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,시험대비특강,4
...,...,...,...
543567,FFFE6EFD-14E1-46A1-B2AF-FB5E2CB3DE2E,수준별영어,133
543568,FFFE6EFD-14E1-46A1-B2AF-FB5E2CB3DE2E,시험대비특강,28
543569,FFFE6EFD-14E1-46A1-B2AF-FB5E2CB3DE2E,실력평가,8
543570,FFFE6EFD-14E1-46A1-B2AF-FB5E2CB3DE2E,첨삭과외,51


In [14]:
menu_list = list(v_raw_d_study['메뉴명'].unique())
menu_list

['매일학교공부',
 '수준별수학',
 '첨삭과외',
 '실력평가',
 '수준별영어',
 '매일 보는 동화',
 '특별학습',
 '중학특강',
 '시험대비특강',
 '키즈 수학',
 '',
 '키즈 한글',
 '키즈 영어',
 '예비초등',
 None]

In [15]:
for menu in menu_list:
    memb_info['메뉴명:{}_수'.format(menu)] = None
    
    if menu==None:
        print("area 0 | menu: {}".format(menu))
        menu_df = pd.DataFrame(v_raw_d_study[v_raw_d_study['메뉴명'].isnull()].groupby(['UserId'])['UserId'].count()).rename(columns={'UserId':'count'})
        menu_df.reset_index(level=['UserId'], inplace=True)
        
        for row in menu_df.values:
            idx = memb_info[memb_info['UserId']==row[0]].index[0]
            memb_info['메뉴명:{}_수'.format(menu)][idx] = row[1]
    else:
        print("area 1 | menu: {}".format(menu))
        for row in menu_df[menu_df['메뉴명']==menu].values:
            idx = memb_info[memb_info['UserId']==row[0]].index[0]
            memb_info['메뉴명:{}_수'.format(menu)][idx] = row[2]

area 1 | menu: 매일학교공부


/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


area 1 | menu: 수준별수학
area 1 | menu: 첨삭과외
area 1 | menu: 실력평가
area 1 | menu: 수준별영어
area 1 | menu: 매일 보는 동화
area 1 | menu: 특별학습
area 1 | menu: 중학특강
area 1 | menu: 시험대비특강
area 1 | menu: 키즈 수학
area 1 | menu: 
area 1 | menu: 키즈 한글
area 1 | menu: 키즈 영어
area 1 | menu: 예비초등
area 0 | menu: None


/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


### 1.4 강의 타입 별 수 추출

In [16]:
lec_type_df = pd.DataFrame(v_raw_d_study.groupby(['UserId', '강의 타입'])['UserId'].count()).rename(columns={'UserId':'count'})
lec_type_df.reset_index(level=['UserId', '강의 타입'], inplace=True)
lec_type_df

,UserId,강의 타입,count
0,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,(영상+문제),333
1,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,문제,4
2,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,영상,169
3,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,첨삭,15
4,0001F20D-B1C7-4101-95A9-B82763380CBE,(영상+문제),707
...,...,...,...
278418,FFFDB3C4-D666-4516-92F7-C6F5BD9721D4,첨삭,47
278419,FFFE6EFD-14E1-46A1-B2AF-FB5E2CB3DE2E,(영상+문제),961
278420,FFFE6EFD-14E1-46A1-B2AF-FB5E2CB3DE2E,문제,26
278421,FFFE6EFD-14E1-46A1-B2AF-FB5E2CB3DE2E,영상,79


In [17]:
lec_type_list = list(v_raw_d_study['강의 타입'].unique())
lec_type_list

['(영상+문제)',
 '첨삭',
 '문제',
 '영상',
 'T_ABILITY_EVAL_SS_OL없음',
 'CORE없음',
 'WHY없음',
 'T_PENG없음']

In [18]:
for lec_type in lec_type_list:
    print("running of preprocessing type: {}".format(lec_type))
    memb_info['강의타입:{}_수'.format(lec_type)] = None
    
    for row in lec_type_df[lec_type_df['강의 타입']==lec_type].values:
        idx = memb_info[memb_info['UserId']==row[0]].index[0]
        memb_info['강의타입:{}_수'.format(lec_type)][idx] = row[2]

running of preprocessing type: (영상+문제)


/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


running of preprocessing type: 첨삭
running of preprocessing type: 문제
running of preprocessing type: 영상
running of preprocessing type: T_ABILITY_EVAL_SS_OL없음
running of preprocessing type: CORE없음
running of preprocessing type: WHY없음
running of preprocessing type: T_PENG없음


### 1.5 타입 별 수 추출

In [19]:
type_df = pd.DataFrame(v_raw_d_study.groupby(['UserId', '타입'])['UserId'].count()).rename(columns={'UserId':'count'})
type_df.reset_index(level=['UserId', '타입'], inplace=True)
type_df

,UserId,타입,count
0,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,순수학습,177
1,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,차시,185
2,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,코스,159
3,0001F20D-B1C7-4101-95A9-B82763380CBE,순수학습,341
4,0001F20D-B1C7-4101-95A9-B82763380CBE,코스,434
...,...,...,...
206559,FFFDB3C4-D666-4516-92F7-C6F5BD9721D4,차시,256
206560,FFFDB3C4-D666-4516-92F7-C6F5BD9721D4,코스,569
206561,FFFE6EFD-14E1-46A1-B2AF-FB5E2CB3DE2E,순수학습,465
206562,FFFE6EFD-14E1-46A1-B2AF-FB5E2CB3DE2E,차시,133


In [20]:
type_list = list(v_raw_d_study['타입'].unique())
type_list

['코스', '순수학습', '차시', None]

In [21]:
for type in type_list:
    memb_info['타입:{}_수'.format(type)] = None
    
    if type==None:
        print("area 0 | type: {}".format(type))
        type_df = pd.DataFrame(v_raw_d_study[v_raw_d_study['타입'].isnull()].groupby(['UserId'])['UserId'].count()).rename(columns={'UserId':'count'})
        type_df.reset_index(level=['UserId'], inplace=True)
        
        for row in type_df.values:
            idx = memb_info[memb_info['UserId']==row[0]].index[0]
            memb_info['타입:{}_수'.format(type)][idx] = row[1]
    else:
        print("area 1 | type: {}".format(type))
        for row in type_df[type_df['타입']==type].values:
            idx = memb_info[memb_info['UserId']==row[0]].index[0]
            memb_info['타입:{}_수'.format(type)][idx] = row[2]

area 1 | type: 코스


/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


area 1 | type: 순수학습
area 1 | type: 차시
area 0 | type: None


/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


## 4. V_RAW_M(Report Table)

### 4.1 connect to DB

In [22]:
# MSSQL connection
conn = pymssql.connect(host=r"",
                       user='',
                       password='',
                       database='')

# Connection으로부터 Cursor 생성
cursor = conn.cursor()

# set query
query = "SELECT [USERID], [집계월], [말일기준 학년], [진도코스], [말일기준 누적 학습기간], [출석횟수], [출석횟수_토, 일 출석 제외] ,[출석율], [완료학습개수], [계획완료개수_코스], [계획완료개수_차시추가], [비계획완료], [순수완료], [풀이문항수(한글)], [풀이문항수(안전한 생활)], [풀이문항수(영어)], [풀이문항수(국어)], [풀이문항수(수학)], [풀이문항수(과학)] ,[풀이문항수(봄여름가을겨울)], [풀이문항수(사회)], [풀이문항수(독서 논술)], [풀이문항수(매일 보는 동화)], [정답수(한글)], [정답수(안전한 생활)], [정답수(영어)], [정답수(국어)], [정답수(수학)], [정답수(과학)], [정답수(봄여름가을겨울)], [정답수(사회)], [정답수(독서 논술)], [정답수(매일 보는 동화)], [오답노트 담긴 강의 수(한글)], [오답노트 담긴 강의 수(안전한 생활)], [오답노트 담긴 강의 수(영어)], [오답노트 담긴 강의 수(국어)], [오답노트 담긴 강의 수(수학)], [오답노트 담긴 강의 수(과학)], [오답노트 담긴 강의 수(봄여름가을겨울)], [오답노트 담긴 강의 수(사회)], [오답노트 담긴 강의 수(독서 논술)], [오답노트 담긴 강의 수(매일 보는 동화)], [학습한 오답노트 강의 수(한글)], [학습한 오답노트 강의 수(안전한 생활)], [학습한 오답노트 강의 수(영어)], [학습한 오답노트 강의 수(국어)], [학습한 오답노트 강의 수(수학)], [학습한 오답노트 강의 수(과학)], [학습한 오답노트 강의 수(봄여름가을겨울)], [학습한 오답노트 강의 수(사회)], [학습한 오답노트 강의 수(독서 논술)], [학습한 오답노트 강의 수(매일 보는 동화)], [점수(한글)], [점수(안전한 생활)], [점수(영어)], [점수(국어)], [점수(수학)], [점수(과학)], [점수(봄여름가을겨울)], [점수(사회)], [점수(독서 논술)], [점수(매일 보는 동화)] FROM [DB].[dbo].[TABLE]"

cursor.execute(query)

# to define columns name
col_names_list = []

for i in range(len(cursor.description)):
    col_names_list.append(cursor.description[i][0])
    
# set query
query = "SELECT [USERID], [집계월], [말일기준 학년], [진도코스], [말일기준 누적 학습기간], [출석횟수], [출석횟수_토, 일 출석 제외] ,[출석율], [완료학습개수], [계획완료개수_코스], [계획완료개수_차시추가], [비계획완료], [순수완료], [풀이문항수(한글)], [풀이문항수(안전한 생활)], [풀이문항수(영어)], [풀이문항수(국어)], [풀이문항수(수학)], [풀이문항수(과학)] ,[풀이문항수(봄여름가을겨울)], [풀이문항수(사회)], [풀이문항수(독서 논술)], [풀이문항수(매일 보는 동화)], [정답수(한글)], [정답수(안전한 생활)], [정답수(영어)], [정답수(국어)], [정답수(수학)], [정답수(과학)], [정답수(봄여름가을겨울)], [정답수(사회)], [정답수(독서 논술)], [정답수(매일 보는 동화)], [오답노트 담긴 강의 수(한글)], [오답노트 담긴 강의 수(안전한 생활)], [오답노트 담긴 강의 수(영어)], [오답노트 담긴 강의 수(국어)], [오답노트 담긴 강의 수(수학)], [오답노트 담긴 강의 수(과학)], [오답노트 담긴 강의 수(봄여름가을겨울)], [오답노트 담긴 강의 수(사회)], [오답노트 담긴 강의 수(독서 논술)], [오답노트 담긴 강의 수(매일 보는 동화)], [학습한 오답노트 강의 수(한글)], [학습한 오답노트 강의 수(안전한 생활)], [학습한 오답노트 강의 수(영어)], [학습한 오답노트 강의 수(국어)], [학습한 오답노트 강의 수(수학)], [학습한 오답노트 강의 수(과학)], [학습한 오답노트 강의 수(봄여름가을겨울)], [학습한 오답노트 강의 수(사회)], [학습한 오답노트 강의 수(독서 논술)], [학습한 오답노트 강의 수(매일 보는 동화)], [점수(한글)], [점수(안전한 생활)], [점수(영어)], [점수(국어)], [점수(수학)], [점수(과학)], [점수(봄여름가을겨울)], [점수(사회)], [점수(독서 논술)], [점수(매일 보는 동화)] FROM [DB].[dbo].[TABLE]"

### 4.2 read to data

In [23]:
# read to data
v_raw_m = pd.DataFrame(list(cursor.fetchall()), columns=col_names_list)

# change to string type: 'USERID' UUID -> string
v_raw_m['USERID'] = v_raw_m['USERID'].astype(str).str.upper().values

### 4.3 preprocessing

In [24]:
# jupyter notebook 상단에서 설정한 start_date & end date를 V_RAW_M에서 where 조건으로 걸기가 제한적
# LIKE문 써도 query상에서 끌어오지 못함: maybe... running time error?
# 따라서, jupyter notebook에서 정제를 해야하며,
# 이에 따라 2019년 이력으로 한정하기 위해 2019년 list 생성
period_2019 = []

for i in range(1, 13):
    if len(str(i))==1:
        period_2019.append('20190{}'.format(i))
    else:
        period_2019.append('2019{}'.format(i))

In [25]:
# 2019년 이력으로 slicing
v_raw_m = v_raw_m[v_raw_m['집계월'].isin(period_2019)]

#### 4.3.1 XX_정답율 계산 및 추출
- 정답율은 다음과 같이 정의됨
- 정답수(XX) / 풀이문항수(XX)

In [26]:
# 파생변수(XX_정답율) 생성에 필요한 변수만 추출
tmp_v_raw_m = v_raw_m[['USERID',
                       '풀이문항수(한글)', '풀이문항수(영어)', '풀이문항수(국어)', '풀이문항수(수학)', '풀이문항수(과학)', '풀이문항수(봄여름가을겨울)', '풀이문항수(사회)', '풀이문항수(독서 논술)',
                       '정답수(한글)', '정답수(영어)', '정답수(국어)', '정답수(수학)', '정답수(과학)', '정답수(봄여름가을겨울)', '정답수(사회)', '정답수(독서 논술)']].groupby('USERID').sum().reset_index(level=['USERID'])

# memb_info에 존재하는 user로만 slicing
tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

# sorting by userid
tmp_v_raw_m.sort_values(by='USERID').reset_index(drop=True)
tmp_v_raw_m

,USERID,풀이문항수(한글),풀이문항수(영어),풀이문항수(국어),풀이문항수(수학),풀이문항수(과학),풀이문항수(봄여름가을겨울),풀이문항수(사회),풀이문항수(독서 논술),정답수(한글),정답수(영어),정답수(국어),정답수(수학),정답수(과학),정답수(봄여름가을겨울),정답수(사회),정답수(독서 논술)
0,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,0.0,0.0,121.0,113.0,0.0,105.0,0.0,0.0,0.0,0.0,83.0,71.0,0.0,69.0,0.0,0.0
1,0001F20D-B1C7-4101-95A9-B82763380CBE,0.0,0.0,477.0,475.0,0.0,430.0,0.0,0.0,0.0,0.0,351.0,348.0,0.0,341.0,0.0,0.0
2,000669AC-C324-43BA-973A-71E31FBA4AEE,0.0,0.0,405.0,487.0,0.0,245.0,5.0,0.0,0.0,0.0,217.0,227.0,0.0,162.0,0.0,0.0
3,000864C3-209F-48C5-9FE8-FBB78F0A34F2,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
4,00088792-7DDF-4D6B-B2BC-FA2F9BCCC211,0.0,0.0,25.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74720,FFFCC9D6-1F60-47DE-8E91-45B094D48624,0.0,5.0,132.0,79.0,58.0,0.0,50.0,0.0,0.0,3.0,103.0,60.0,38.0,0.0,36.0,0.0
74721,FFFD15A5-6778-429A-9115-78A6012CCB6F,0.0,0.0,77.0,84.0,5.0,50.0,5.0,0.0,0.0,0.0,64.0,71.0,2.0,41.0,3.0,0.0
74722,FFFD7651-C51D-4912-8AA9-9E93A75D8D2D,0.0,820.0,570.0,499.0,319.0,5.0,251.0,16.0,0.0,650.0,516.0,369.0,252.0,5.0,196.0,12.0
74723,FFFDB3C4-D666-4516-92F7-C6F5BD9721D4,0.0,545.0,964.0,1005.0,332.0,30.0,352.0,0.0,0.0,292.0,603.0,671.0,180.0,24.0,200.0,0.0


In [27]:
sub_list = ['한글', '영어', '국어', '수학', '봄여름가을겨울', '매일 보는 동화', '안전한 생활', '독서 논술', '과학', '사회']
sub_list

['한글', '영어', '국어', '수학', '봄여름가을겨울', '매일 보는 동화', '안전한 생활', '독서 논술', '과학', '사회']

In [28]:
for subject in sub_list:
    try:
        memb_info['{}_정답율'.format(subject)] = round(tmp_v_raw_m['정답수({})'.format(subject)] / tmp_v_raw_m['풀이문항수({})'.format(subject)], 2) * 100
        
    # group by 시 해당 subject(column)의 row가 모두 null이면 해당 column은 group by되지 않기 때문에
    # keyError를 발생시킴
    except KeyError:
        print("###### [KeyError] where the occurred: {}".format(subject))

###### [KeyError] where the occurred: 매일 보는 동화
###### [KeyError] where the occurred: 안전한 생활


#### 4.3.2 XX_오답_수행율 계산 및 추출
- 오답_수행율은 다음과 같이 정의됨
- 학습한 오답노트 강의 수(XX) / 오답노트 담긴 강의 수(XX)

In [29]:
# 파생변수(XX_오답_수행율) 생성에 필요한 변수만 추출
tmp_v_raw_m = v_raw_m[['USERID',
                       '오답노트 담긴 강의 수(한글)', '오답노트 담긴 강의 수(영어)', '오답노트 담긴 강의 수(국어)', '오답노트 담긴 강의 수(수학)', '오답노트 담긴 강의 수(과학)', '오답노트 담긴 강의 수(봄여름가을겨울)', '오답노트 담긴 강의 수(사회)', '오답노트 담긴 강의 수(독서 논술)',
                       '학습한 오답노트 강의 수(한글)', '학습한 오답노트 강의 수(영어)', '학습한 오답노트 강의 수(국어)', '학습한 오답노트 강의 수(수학)', '학습한 오답노트 강의 수(과학)', '학습한 오답노트 강의 수(봄여름가을겨울)', '학습한 오답노트 강의 수(사회)', '학습한 오답노트 강의 수(독서 논술)']].groupby('USERID').sum().reset_index(level=['USERID'])

# memb_info에 존재하는 user로만 slicing
tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

# sorting by userid
tmp_v_raw_m.sort_values(by='USERID').reset_index(drop=True)
tmp_v_raw_m

,USERID,오답노트 담긴 강의 수(한글),오답노트 담긴 강의 수(영어),오답노트 담긴 강의 수(국어),오답노트 담긴 강의 수(수학),오답노트 담긴 강의 수(과학),오답노트 담긴 강의 수(봄여름가을겨울),오답노트 담긴 강의 수(사회),오답노트 담긴 강의 수(독서 논술),학습한 오답노트 강의 수(한글),학습한 오답노트 강의 수(영어),학습한 오답노트 강의 수(국어),학습한 오답노트 강의 수(수학),학습한 오답노트 강의 수(과학),학습한 오답노트 강의 수(봄여름가을겨울),학습한 오답노트 강의 수(사회),학습한 오답노트 강의 수(독서 논술)
0,0000B68D-EBC2-40FA-985D-A3454DDEFFA7,0.0,0.0,47.0,53.0,0.0,47.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
1,0001F20D-B1C7-4101-95A9-B82763380CBE,0.0,0.0,120.0,129.0,0.0,86.0,0.0,0.0,0.0,0.0,45.0,58.0,0.0,27.0,0.0,0.0
2,000669AC-C324-43BA-973A-71E31FBA4AEE,0.0,0.0,200.0,270.0,0.0,90.0,5.0,0.0,0.0,0.0,145.0,140.0,0.0,45.0,0.0,0.0
3,000864C3-209F-48C5-9FE8-FBB78F0A34F2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00088792-7DDF-4D6B-B2BC-FA2F9BCCC211,0.0,0.0,6.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74720,FFFCC9D6-1F60-47DE-8E91-45B094D48624,0.0,2.0,37.0,24.0,22.0,0.0,18.0,0.0,0.0,0.0,9.0,10.0,3.0,0.0,1.0,0.0
74721,FFFD15A5-6778-429A-9115-78A6012CCB6F,0.0,0.0,17.0,16.0,3.0,11.0,2.0,0.0,0.0,0.0,16.0,15.0,3.0,11.0,2.0,0.0
74722,FFFD7651-C51D-4912-8AA9-9E93A75D8D2D,0.0,11.0,54.0,133.0,69.0,0.0,56.0,0.0,0.0,1.0,12.0,24.0,36.0,0.0,10.0,0.0
74723,FFFDB3C4-D666-4516-92F7-C6F5BD9721D4,0.0,177.0,381.0,410.0,149.0,8.0,171.0,0.0,0.0,110.0,248.0,257.0,110.0,0.0,119.0,0.0


In [30]:
for subject in sub_list:
    try:
        memb_info['{}_오답_수행율'.format(subject)] = round(tmp_v_raw_m['학습한 오답노트 강의 수({})'.format(subject)] / tmp_v_raw_m['오답노트 담긴 강의 수({})'.format(subject)], 2) * 100
        
    # group by 시 해당 subject(column)의 row가 모두 null이면 해당 column은 group by되지 않기 때문에
    # keyError를 발생시킴
    except KeyError:
        print("###### [KeyError] where the occurred: {}".format(subject))

###### [KeyError] where the occurred: 매일 보는 동화
###### [KeyError] where the occurred: 안전한 생활


#### 4.3.3 XX_평균_점수 계산 및 추출
- 평균_점수는 다음과 같이 정의됨
- SUM(XX_점수) / len(집계월)

In [31]:
# 파생변수(XX_평균_점수) 생성에 필요한 변수만 추출
tmp_v_raw_m = v_raw_m[['USERID',
                       '점수(한글)', '점수(영어)', '점수(국어)', '점수(수학)', '점수(과학)', '점수(봄여름가을겨울)', '점수(사회)', '점수(독서 논술)']]

# memb_info에 존재하는 user로만 slicing
tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

# sorting by userid
tmp_v_raw_m.sort_values(by='USERID').reset_index(drop=True)
tmp_v_raw_m

,USERID,점수(한글),점수(영어),점수(국어),점수(수학),점수(과학),점수(봄여름가을겨울),점수(사회),점수(독서 논술)
0,34DDF9B4-08F2-4414-9023-06B6FAD09A46,NaN,NaN,70.0,58.0,80.0,NaN,60.0,NaN
1,9070C4D0-2FF6-4BDB-AB5D-90F4B3C65466,NaN,90.0,100.0,100.0,NaN,100.0,NaN,NaN
2,00EF3F15-366B-4D62-BEFE-544797ABEC1E,NaN,NaN,93.0,83.0,80.0,NaN,87.0,NaN
3,21125B18-17C6-403D-B665-ED667E727288,NaN,NaN,77.0,74.0,90.0,NaN,50.0,NaN
4,CFF28889-3328-4F74-B74C-83D78E7A1780,NaN,90.0,86.0,76.0,77.0,NaN,93.0,NaN
...,...,...,...,...,...,...,...,...,...
655992,1BBE2327-AE22-4C2B-9D47-C0D8E5BB7BBA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
655993,6D2F43AA-A462-4C17-A5F8-BF2296E99C94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
655994,E32B52DD-A13C-400A-81D3-3F36D652F7FE,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN
655995,C7F26B40-4BBA-4D76-ACDA-B74E812A4029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
for subject in sub_list:
    try:
        memb_info['{}_평균_점수'.format(subject)] = tmp_v_raw_m.groupby(['USERID'])['점수({})'.format(subject)].mean().values
    except KeyError:
        print("###### [KeyError] where the occurred: ".format(subject))

###### [KeyError] where the occurred: 
###### [KeyError] where the occurred: 


#### 4.3.4 나머지 정보들 추출(학년, 출석 관련, ...)

In [33]:
# 말일기준 학년
memb_info['학년'] = None

tmp_v_raw_m = v_raw_m.groupby(['USERID'])['말일기준 학년'].min().reset_index(level=['USERID'])

tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

for row in tmp_v_raw_m.values:
    idx = memb_info[memb_info['UserId']==row[0]].index[0]
    memb_info['학년'][idx] = row[1]

/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [34]:
# 말일기준 누적 학습기간
memb_info['누적학습기간'] = None

tmp_v_raw_m = v_raw_m.groupby(['USERID'])['말일기준 누적 학습기간'].max().reset_index(level=['USERID'])

tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

for row in tmp_v_raw_m.values:
    idx = memb_info[memb_info['UserId']==row[0]].index[0]
    memb_info['누적학습기간'][idx] = row[1]

/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [35]:
# 출석횟수
memb_info['출석횟수'] = None

tmp_v_raw_m = v_raw_m.groupby(['USERID'])['출석횟수'].sum().reset_index(level=['USERID'])

tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

for row in tmp_v_raw_m.values:
    idx = memb_info[memb_info['UserId']==row[0]].index[0]
    memb_info['출석횟수'][idx] = row[1]

/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [36]:
# 주중출석횟수
memb_info['주중출석횟수'] = None

tmp_v_raw_m = v_raw_m.groupby(['USERID'])['출석횟수_토, 일 출석 제외'].sum().reset_index(level=['USERID'])

tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

for row in tmp_v_raw_m.values:
    idx = memb_info[memb_info['UserId']==row[0]].index[0]
    memb_info['주중출석횟수'][idx] = row[1]

/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [37]:
# 출석율
memb_info['평균출석율'] = None

tmp_v_raw_m = v_raw_m.groupby(['USERID'])['출석율'].mean().reset_index(level=['USERID'])

tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

for row in tmp_v_raw_m.values:
    idx = memb_info[memb_info['UserId']==row[0]].index[0]
    memb_info['평균출석율'][idx] = row[1]

/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [38]:
# 완료학습개수
memb_info['완료학습개수'] = None

tmp_v_raw_m = v_raw_m.groupby(['USERID'])['완료학습개수'].sum().reset_index(level=['USERID'])

tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

for row in tmp_v_raw_m.values:
    idx = memb_info[memb_info['UserId']==row[0]].index[0]
    memb_info['완료학습개수'][idx] = row[1]

/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [39]:
# 계획완료개수_코스
memb_info['계획완료개수_코스'] = None

tmp_v_raw_m = v_raw_m.groupby(['USERID'])['계획완료개수_코스'].sum().reset_index(level=['USERID'])

tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

for row in tmp_v_raw_m.values:
    idx = memb_info[memb_info['UserId']==row[0]].index[0]
    memb_info['계획완료개수_코스'][idx] = row[1]

/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [40]:
# 계획완료개수_차시추가
memb_info['계획완료개수_차시추가'] = None

tmp_v_raw_m = v_raw_m.groupby(['USERID'])['계획완료개수_차시추가'].sum().reset_index(level=['USERID'])

tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

for row in tmp_v_raw_m.values:
    idx = memb_info[memb_info['UserId']==row[0]].index[0]
    memb_info['계획완료개수_차시추가'][idx] = row[1]

/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [41]:
# 비계획완료
memb_info['비계획완료'] = None

tmp_v_raw_m = v_raw_m.groupby(['USERID'])['비계획완료'].sum().reset_index(level=['USERID'])

tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

for row in tmp_v_raw_m.values:
    idx = memb_info[memb_info['UserId']==row[0]].index[0]
    memb_info['비계획완료'][idx] = row[1]

/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [42]:
# 순수완료
memb_info['순수완료'] = None

tmp_v_raw_m = v_raw_m.groupby(['USERID'])['순수완료'].sum().reset_index(level=['USERID'])

tmp_v_raw_m = tmp_v_raw_m[tmp_v_raw_m['USERID'].isin(memb_info['UserId'].unique())].reset_index(drop=True)

for row in tmp_v_raw_m.values:
    idx = memb_info[memb_info['UserId']==row[0]].index[0]
    memb_info['순수완료'][idx] = row[1]

/data/user/venv/model-irt/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


# 5. export to data

In [43]:
# 1~6학년으로만 대상 한정
memb_info = memb_info[memb_info['학년'].isin([1, 2, 3, 4, 5, 6])].reset_index(drop=True)

In [44]:
memb_info[['UserId', 'Gender', '학년', 'capital_area', 'charge_memb',
           '누적학습기간', '출석횟수', '주중출석횟수', '평균출석율',
           '완료학습개수', '계획완료개수_코스', '계획완료개수_차시추가', '비계획완료', '순수완료',
           '메뉴명:매일학교공부_수', '메뉴명:중학특강_수', '메뉴명:첨삭과외_수', '메뉴명:수준별수학_수',
           '메뉴명:실력평가_수', '메뉴명:수준별영어_수', '메뉴명:특별학습_수', '메뉴명:매일 보는 동화_수',
           '메뉴명:시험대비특강_수', '메뉴명:키즈 영어_수', '메뉴명:_수', '메뉴명:키즈 한글_수',
           '메뉴명:키즈 수학_수', '메뉴명:예비초등_수', '메뉴명:None_수',
           '강의타입:(영상+문제)_수', '강의타입:첨삭_수', '강의타입:문제_수', '강의타입:영상_수',
           '강의타입:T_ABILITY_EVAL_SS_OL없음_수', '강의타입:CORE없음_수', '강의타입:WHY없음_수', '강의타입:T_PENG없음_수',
           '타입:코스_수', '타입:순수학습_수', '타입:차시_수', '타입:None_수',
           '한글_정답율', '국어_정답율', '영어_정답율', '수학_정답율',  '사회_정답율', '과학_정답율', '봄여름가을겨울_정답율', '독서 논술_정답율',
           '한글_오답_수행율', '국어_오답_수행율', '영어_오답_수행율', '수학_오답_수행율', '사회_오답_수행율', '과학_오답_수행율', '봄여름가을겨울_오답_수행율',
           '한글_평균_점수', '국어_평균_점수', '영어_평균_점수', '수학_평균_점수', '사회_평균_점수', '과학_평균_점수', '봄여름가을겨울_평균_점수', '독서 논술_평균_점수']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66544 entries, 0 to 66543
Data columns (total 64 columns):
UserId                           66544 non-null object
Gender                           66544 non-null object
학년                               66544 non-null object
capital_area                     66519 non-null object
charge_memb                      66544 non-null int64
누적학습기간                           66499 non-null object
출석횟수                             66544 non-null object
주중출석횟수                           66544 non-null object
평균출석율                            66543 non-null object
완료학습개수                           66544 non-null object
계획완료개수_코스                        66544 non-null object
계획완료개수_차시추가                      66544 non-null object
비계획완료                            66544 non-null object
순수완료                             66544 non-null object
메뉴명:매일학교공부_수                     66387 non-null object
메뉴명:중학특강_수                       8886 non-null object
메뉴명:첨삭과외_수   

In [46]:
memb_info[['UserId', 'Gender', '학년', 'capital_area', 'charge_memb',
           '누적학습기간', '출석횟수', '주중출석횟수', '평균출석율',
           '완료학습개수', '계획완료개수_코스', '계획완료개수_차시추가', '비계획완료', '순수완료',
           '메뉴명:매일학교공부_수', '메뉴명:중학특강_수', '메뉴명:첨삭과외_수', '메뉴명:수준별수학_수',
           '메뉴명:실력평가_수', '메뉴명:수준별영어_수', '메뉴명:특별학습_수', '메뉴명:매일 보는 동화_수',
           '메뉴명:시험대비특강_수', '메뉴명:키즈 영어_수', '메뉴명:_수', '메뉴명:키즈 한글_수',
           '메뉴명:키즈 수학_수', '메뉴명:예비초등_수', '메뉴명:None_수',
           '강의타입:(영상+문제)_수', '강의타입:첨삭_수', '강의타입:문제_수', '강의타입:영상_수',
           '강의타입:T_ABILITY_EVAL_SS_OL없음_수', '강의타입:CORE없음_수', '강의타입:WHY없음_수', '강의타입:T_PENG없음_수',
           '타입:코스_수', '타입:순수학습_수', '타입:차시_수', '타입:None_수',
           '한글_정답율', '국어_정답율', '영어_정답율', '수학_정답율',  '사회_정답율', '과학_정답율', '봄여름가을겨울_정답율', '독서 논술_정답율',
           '한글_오답_수행율', '국어_오답_수행율', '영어_오답_수행율', '수학_오답_수행율', '사회_오답_수행율', '과학_오답_수행율', '봄여름가을겨울_오답_수행율',
           '한글_평균_점수', '국어_평균_점수', '영어_평균_점수', '수학_평균_점수', '사회_평균_점수', '과학_평균_점수', '봄여름가을겨울_평균_점수', '독서 논술_평균_점수']].to_csv('/data/user/workspace/model-irt/memb_info_v1.0.csv',
                                                                                                                           header=True,
                                                                                                                           index=False)